In [5]:
import torch
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data import DataLoader
import torch.nn.functional as F
# （bahchsize, 1，28，28）--> (卷积:  卷积核 5*5,输出(batchsize ,10,24,24) --> maxpooling(2,2) --> relu(batchsize , 10,12,12) -> 
#  卷积(内核 5 * 5, 输出 (batchsize,20,8,8)) ->   maxpooling(2,2) - relu(batchsize , 20,4,4) -> linearn(20 * 4 * 4 = 320) --> 10 

In [80]:
class CnnNet(torch.nn.Module):
    def __init__(self):
        super(CnnNet,self).__init__()
        self.conv_1 = torch.nn.Conv2d(1,10,kernel_size=(5,5))
        self.conv_2 = torch.nn.Conv2d(10,20,kernel_size=(5,5))
        self.pooling = torch.nn.MaxPool2d((2,2))
        self.linear = torch.nn.Linear(320,10)
    
    def forward(self,x):
        batch_size = x.size(0)
        x = F.relu(self.pooling(self.conv_1(x)))
#         print(x.shape)
        x = F.relu(self.pooling(self.conv_2(x)))
        # 变成一列
#         print(x.shape)
        x = x.view(batch_size,-1)
#         print(x.shape)
        x = self.linear(x)
        return x
    


In [8]:
transform = transforms.Compose([
    transforms.ToTensor(),
#     transforms.Normalize()
    ])
train_data = datasets.FashionMNIST(root="ai_data",train=True,download=False,transform=transform)
test_data = datasets.FashionMNIST(root="ai_data",train=False,download=False,transform=transform)
train_loder = DataLoader(train_data,batch_size=32,shuffle=True,num_workers=1)
test_loder = DataLoader(train_data,batch_size=32,shuffle=True,num_workers=1)

In [81]:
cnn_net = CnnNet()
loss = torch.nn.CrossEntropyLoss(reduction="mean")
optimizer = torch.optim.SGD(cnn_net.parameters(),lr=0.01,momentum=0.5)

In [131]:
def train(epochs):
    num = []
    loss_item = []
    running_loss = 0.0
    for epoch in range(epochs):
        
        for i , data in enumerate(train_loder):
            x, y = data
            y_hat = cnn_net(x)
            l = loss(y_hat , y)
            optimizer.zero_grad()
            l.backward()
            optimizer.step()
#             print(f"第{epoch + 1}轮 ， 第{i}次的损失值:{l.item()}")
            
            running_loss += l.item()
            # 每 100 次 查看一下损失
            if i % 50 == 49:
                num.append(i+1)
                loss_item.append(running_loss / 100)
                running_loss = 0.0
#                 break
    print(num)
    print(loss_item)
train(1)
test()

[50, 100, 150, 200, 250, 300, 350, 400, 450, 500, 550, 600, 650, 700, 750, 800, 850, 900, 950, 1000, 1050, 1100, 1150, 1200, 1250, 1300, 1350, 1400, 1450, 1500, 1550, 1600, 1650, 1700, 1750, 1800, 1850]
[0.21345706805586814, 0.20614102944731713, 0.21352418914437293, 0.22124315947294235, 0.21388401389122008, 0.2355054000020027, 0.20451712988317014, 0.20971192330121993, 0.23094988137483596, 0.2001102887094021, 0.22219557493925093, 0.21251028448343276, 0.19906586319208144, 0.2031318140029907, 0.2114003126323223, 0.20733403727412225, 0.20513707712292673, 0.21196244686841964, 0.1953221020102501, 0.1774888701736927, 0.216834122389555, 0.20997752249240875, 0.21641527652740478, 0.21529632434248924, 0.2069263057410717, 0.18000113785266877, 0.199957125633955, 0.20526699870824813, 0.195643662661314, 0.1973426252603531, 0.19907417759299278, 0.20998738780617715, 0.21577485859394074, 0.19023045763373375, 0.19888735190033913, 0.1952510331571102, 0.22179043605923654]
Accuracy on test set: 96 %


In [123]:
def test():
    corrent = 0
    total = 0
    with torch.no_grad():
        for i , data in enumerate(train_loder):
            x , y = data
            y_hat = cnn_net(x)

            '''
            torch.max(input, dim, keepdim=False, out=None)
            按维度dim返回最大值，0为按列取最大值，1为按行取最大值
            （最大值，最大值索引）
            下面语句代表按行取最大值对应索引，即标签值        
            prediction[0] 代表取值， [1]代表取对应的索引
            '''
            prediction = torch.max(y_hat.data, 1)[1]
            total += len(y)
            corrent += (y == prediction).sum().item()
            break
        print('Accuracy on test set: %d %%' % (100 * corrent / total))
test()
    

Accuracy on test set: 65 %


In [119]:
def start():
    # 训练3轮
    train(1)
    test()
start()
        

0
1
2


299